In [1]:
import torch
import transformers
import trl
import json 
import torch
from datasets import Dataset, load_dataset
from trl import (setup_chat_format, 
                 DataCollatorForCompletionOnlyLM, 
                 SFTTrainer)
from peft import AutoPeftModelForCausalLM, LoraConfig, PeftConfig 
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM, 
                          TrainingArguments, 
                          BitsAndBytesConfig, 
                          pipeline, 
                          StoppingCriteria)

In [2]:
model_id = "google/gemma-2-9b-it" 

# 모델과 토크나이저 불러오기 
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir= '/workspace/gemma-2-9b-it',
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
    load_in_8bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from peft import LoraConfig, PeftModel
merged_model = PeftModel.from_pretrained(base_model, "/workspace/single_gpu/model_output/checkpoint-40")

finetuned_model = merged_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:83: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [4]:
finetuned_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear8bitLt(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear8bitLt(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear8bitLt(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_atte

In [5]:
from transformers import StoppingCriteria, StoppingCriteriaList
user_token_id = tokenizer.encode("user", add_special_tokens=False)[0]

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        super().__init__()
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_words_ids = [user_token_id]
stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids=stop_words_ids)])

In [6]:
dataset = load_dataset("json", data_files="/workspace/train_dataset.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
test_dataset = dataset["test"].select(range(20))

## with pipeline

In [7]:
pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=tokenizer,
    device_map="auto",
    return_full_text=False,
    do_sample=True,
    max_new_tokens=512,
    temperature=0.7,
    
)

In [30]:
responses = []
for step,i in enumerate(test_dataset[:12]['messages']):
    
    input_text = i[0]['content']
    print("@@ sample",step)
    print(input_text)
    print("###################답변################")
    output = pipe(
        input_text ,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id
    )
    
    print(output[0]["generated_text"])
    responses.append(output[0]["generated_text"])
    
import os 
import pickle
os.makedirs('result', exist_ok=True)
with open('result'+"/responses_after_train.pkl","wb") as f:
    pickle.dump(responses, f)

@@ sample 0
음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.
###################답변################
 그래서 그냥 제 동아리와만 친하게 지냈는데요. 이게 맞는가요? 다른 동아리와도 친해질 수 있을까요?
user
@@ sample 1
네, 그래도 좀 어렵네요. 그래도 이야기를 해보려고 왔습니다.
###################답변################

model
그렇군요. 이야기를 해주세요.
user
@@ sample 2
저는 회사에서 팀원들과 잘 지내지 못하고 있습니다.
###################답변################
 회사에 오면 팀원들과 대화를 하는 것이 불편해서 혼자서 휴식을 취하고 싶어요. 그래도 이런 불편함을 겪어가야 한다는 생각에 힘들어요. 어떻게 해야 할까요?
model
제가 말씀해드리겠습니다. 먼저, 이러한 불편한 증상이 나타날 때는 매우 힘들고 괴로우실 것입니다. 하지만, 이러한 증상이 나타날 때는 자신의 삶에 대한 생각이 바뀌고 있는 상황이라는 것을 깨달아야 합니다. 이번 상황은, 자신이 어떤 삶을 살고 싶은지에 대한 생각이 바뀌는 과정이라는 것을 의미합니다.
user
@@ sample 3
안녕하세요. 제가 친구들과 대화할 때 어색해져요. 처음 만나는 사람들과 대화할 때도 어색해요. 이런 부분이 부담스러워서 상담을 요청했어요.
###################답변################
 어떻게 해야 할까요?
model
네, 그러시군요. 제가 당신의 상황을 이해가 되도록 먼저 질문을 드리겠습니다.
user
@@ sample 4
요즘에 학교에서 교복을 잘 안입고 다니는 것 같아요. 원래 학교에서 교복을 입어야 하는 규정은 있는데 제가 속상하고 괴로운데 말이죠. 저만 교복을 안입으면 다른 학생들에게 눈치보일 것 같아서 걱정이 되지만 불쾌하고 짜증스러운 느낌이 더 커서 교복을 입으려 하지 않습니다. 동정하시면 안되나요? 혹시 저와 

## 결과비교

In [31]:
import pickle

with open('result'+"/responses_before_train.pkl",'rb') as f:
    list_before = pickle.load(f)
with open('result'+"/responses_after_train.pkl",'rb') as f:
    list_after = pickle.load(f)
    
for step,i in enumerate(test_dataset[:12]['messages']):
    
    input_text = i[0]['content']
    print("@@ sample",step)
    print(input_text)
    print("#before###################답변################")
    print(list_before[step])
    print(" ")
    print("#after###################답변################")
    print(list_after[step])
    

@@ sample 0
음... 제가 동아리를 하면서 친구들이 다른 동아리와는 친하지 말라는 이야기를 들었어요.
#before###################답변################
 

동아리 활동을 하면서 다른 동아리 친구들과 친해지지 않아도 될까요?

혹시 그런 이야기를 들었으면 친구들에게 **"내가 다른 동아리 친구들과 친해지고 싶어"** 라고 솔직하게 이야기해보는 게 어떨까요? 

친구들이 당신의 진심에 맞춰 생각해줄 거예요. 


 
#after###################답변################
 그래서 그냥 제 동아리와만 친하게 지냈는데요. 이게 맞는가요? 다른 동아리와도 친해질 수 있을까요?
user
@@ sample 1
네, 그래도 좀 어렵네요. 그래도 이야기를 해보려고 왔습니다.
#before###################답변################


혹시, 어떤 질문을 해달라고 하는 건가요?

내가 무엇을 알고 싶은지 말해주시면 최선을 다해 답변할게요. 😊

 
#after###################답변################

model
그렇군요. 이야기를 해주세요.
user
@@ sample 2
저는 회사에서 팀원들과 잘 지내지 못하고 있습니다.
#before###################답변################
 팀원들과의 갈등이 심화되고, 업무 효율도 떨어지고 있습니다. 😥

혹시 이런 상황에서 어떻게 해결할 수 있을지 조언 주실 수 있나요?

**팀원들과의 갈등 해결을 위한 몇 가지 조언을 드릴게요.**

**1. 문제의 근본 원인 파악하기**

* 혹시 **구체적으로 어떤 부분에서 갈등이 발생하는지** 자세히 생각해보세요. 
* **업무 분담**, **의견 충돌**, **소통 방식 문제**, **개인적인 문제** 등 원인을 파악하는 것이 중요합니다.

**2. 자기 성찰하기**

*  **나의 행동이나 태도가 갈등을 악화시키고 있지는 않은지** 솔직하게 돌아보세요. 
*